Zero shot classification

In [23]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()  # Load environment variables from .env file, contains personal access token (HF_API_TOKEN=your_token)

API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-mnli"
# API_URL = "https://api-inference.huggingface.co/models/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
# API_URL = "https://api-inference.huggingface.co/models/cross-encoder/nli-deberta-v3-base"
# API_URL = "https://api-inference.huggingface.co/models/valhalla/distilbart-mnli-12-3"
headers = {"Authorization": f"Bearer {os.getenv('HF_API_TOKEN')}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()



In [24]:
# Input text to classify
input_text = "I just bought a new laptop, and it works amazing!"

# Candidate labels
candidate_labels = ["technology", "sports", "politics", "health"]

# Get the prediction
output = query({"inputs": input_text, "parameters": {"candidate_labels": candidate_labels}})
print(output)


{'sequence': 'I just bought a new laptop, and it works amazing!', 'labels': ['technology', 'health', 'sports', 'politics'], 'scores': [0.9709171652793884, 0.014999167993664742, 0.008272457867860794, 0.005811102222651243]}


In [25]:
# Input text to classify
input_text = "I like to cycle and I burn easily. I also love culture and like to post on social media about my food. I will go on a trip to italy in july."

# Candidate labels
candidate_labels = [
    "Swimsuit", "Sunscreen", "Flip-flops", "Beach towel", "Sunglasses", 
    "Waterproof phone case", "Hat", "Beach bag", "Snorkel gear", "Aloe vera gel",
    "Tent", "Sleeping bag", "Camping stove", "Flashlight", "Hiking boots",
    "Water filter", "Compass", "First aid kit", "Bug spray", "Multi-tool",
    "Thermal clothing", "Ski jacket", "Ski goggles", "Snow boots", "Gloves",
    "Hand warmers", "Beanie", "Lip balm", "Snowboard", "Base layers",
    "Passport", "Visa documents", "Travel adapter", "Currency", "Language phrasebook",
    "SIM card", "Travel pillow", "Neck wallet", "Travel insurance documents", "Power bank",
    "Laptop", "Notebook", "Business attire", "Dress shoes", "Charging cables",
    "Presentation materials", "Work ID badge", "Pen", "Headphones", 
    "Lightweight backpack", "Travel-sized toiletries", "Packable rain jacket",
    "Reusable water bottle", "Dry bag", "Trekking poles", "Hostel lock", "Quick-dry towel",
    "Travel journal", "Energy bars", "Car charger", "Snacks", "Map",
    "Sunglasses", "Cooler", "Blanket", "Emergency roadside kit", "Reusable coffee mug",
    "Playlist", "Reusable shopping bags", "Earplugs", "Fanny pack", "Portable charger",
    "Poncho", "Bandana", "Comfortable shoes", "Tent", "Refillable water bottle",
    "Glow sticks", "Festival tickets", "Diapers", "Baby wipes", "Baby food",
    "Stroller", "Pacifier", "Baby clothes", "Baby blanket", "Travel crib",
    "Toys", "Nursing cover"
]


# Get the prediction
output = query({"inputs": input_text, "parameters": {"candidate_labels": candidate_labels}})
print(output)

{'error': ['Error in `parameters.candidate_labels`: ensure this value has at most 10 items']}
